In [ ]:
"""
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
"""

In [ ]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
name   ="EURUSD_M1_23yil_DataFrame"
#df=pd.read_csv("EURUSD.csv")
df = pd.read_csv(name+'.csv', engine='python')
    
print("Number of rows and columns:", df.shape)
#df = df.drop('Adj Close',axis=1)
#df = df.drop('Volume',axis=1)
import dill
dill.dump_session(name+'.db')
df.tail(10)

In [ ]:
df = df.drop('Date',axis=1)
train_size = ((len(df)*90)//100)
training_set = df.iloc[:train_size, 1:2].values
test_set = df.iloc[train_size:, 1:2].values

train_size,training_set[:5],test_set[:5]

In [ ]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

time_steps_train = 2

# Creating a data structure with 2 time-steps and 1 output
X_train = []
y_train = []
for i in range(time_steps_train, train_size):
    X_train.append(training_set_scaled[i-time_steps_train:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#(740, 60, 1)

In [ ]:
# Getting the predicted
dataset_train = df.iloc[:train_size, 1:2]
dataset_test = df.iloc[train_size:, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_steps_train:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
y_test = []
for i in range(time_steps_train, len(df)-train_size+time_steps_train): #519
    X_test.append(inputs[i-time_steps_train:i, 0])
    y_test.append(inputs[i, 0])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test.shape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
print(X_test.shape,y_test.shape)
#(459, 60, 1)

In [ ]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set

In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size = 4096)

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualising the results
plt.plot(df.loc[train_size:, "Index"],dataset_test.values, color = "red", label = "Real Price")
plt.plot(df.loc[train_size:, "Index"],predicted_stock_price, color = "blue", label = "Predicted Price")
#plt.xticks(np.arange(0,len(df),50))
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()